In [51]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory



In [52]:
# #import warnings filter
# from warnings import simplefilter
# # ignore all future warnings
# simplefilter(action='ignore', category=FutureWarning)

In [53]:
#Load the sentiment_values.csv - The smallest dataset
df = pd.read_csv('../data/chelseapower/fixed_sentiment_values.csv')

#Look at size of the dataset
df = df[['hashtag', 'ol_score', 'ol_avg']]
df.apply(lambda x: x.isnull().sum())

hashtag        0
ol_score    1423
ol_avg      2467
dtype: int64

In [54]:
#Fill in missing ol_score with ol_avg score, if available
df['ol_score'] = df.apply(
    lambda row: row['ol_avg'] if np.isnan(row['ol_score']) else row['ol_score'],
    axis=1
)
df.apply(lambda x: x.isnull().sum())

hashtag        0
ol_score     459
ol_avg      2467
dtype: int64

In [55]:
df1 = df.dropna(axis = 0, how ='any') 
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2823 entries, 1 to 5289
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   hashtag   2823 non-null   object 
 1   ol_score  2823 non-null   float64
 2   ol_avg    2823 non-null   float64
dtypes: float64(2), object(1)
memory usage: 88.2+ KB


In [56]:
# df1.sort_index(by='hashtag', ascending=[False])

In [57]:
#Rename column
df1.rename(columns = {'ol_score':'sentiment_score'}, inplace = True)
df1.head()

E:\ssdapps\anaconda\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,hashtag,sentiment_score,ol_avg
1,melovechilicheese,0.8,1.0
2,greatmusic,2.4,1.0
4,amonamarth,0.3,0.0
5,againbecauseitissogood,0.7,1.0
6,cloudnine,0.0,0.0


In [58]:
# #Show top 10 hashtags with the largest sentiment_score
# x = df1.nlargest(10, 'sentiment_score')
# x

In [59]:
#Load second dataset
df2 = pd.read_csv('../data/chelseapower/user_track_hashtag_timestamp.csv')

#Look at size of the dataset
df2.shape

(17560113, 4)

In [60]:
#Look at the columns and initial rows of the dataset
df2.head()

,user_id,track_id,hashtag,created_at
0,81496937,cd52b3e5b51da29e5893dba82a418a4b,nowplaying,2014-01-01 05:54:21
1,81496937,cd52b3e5b51da29e5893dba82a418a4b,goth,2014-01-01 05:54:21
2,81496937,cd52b3e5b51da29e5893dba82a418a4b,deathrock,2014-01-01 05:54:21
3,81496937,cd52b3e5b51da29e5893dba82a418a4b,postpunk,2014-01-01 05:54:21
4,2205686924,da3110a77b724072b08f231c9d6f7534,NowPlaying,2014-01-01 05:54:22


In [61]:
#Check for null values
df2.apply(lambda x: x.isnull().sum())

user_id       0
track_id      0
hashtag       1
created_at    0
dtype: int64

In [62]:
#Drop null rows
df2 = df2.dropna(subset=['hashtag'])

In [63]:
# Get the count of the track_id
counts = df2['track_id'].value_counts()

# Select the items where the track_id count is less than 50 and remove them
df2 = df2[~df2['track_id'].isin(counts[counts < 50].index)]

# Show info
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14966614 entries, 0 to 17560112
Data columns (total 4 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   user_id     int64 
 1   track_id    object
 2   hashtag     object
 3   created_at  object
dtypes: int64(1), object(3)
memory usage: 570.9+ MB


In [64]:
#Merge CSV files into a single file based on hashtag
df_sentiment = pd.merge(df1, df2, on="hashtag", how='inner')
df_sentiment.head()

,hashtag,sentiment_score,ol_avg,user_id,track_id,created_at
0,greatmusic,2.4,1.0,252330820,8f2ac86abb8bd48273c8fc95b632e347,2014-02-13 16:18:51
1,greatmusic,2.4,1.0,29235188,45bbd6d7cd65dc77596af8c5c0b89a70,2014-05-15 20:38:46
2,greatmusic,2.4,1.0,2308582644,fca4f99723ab8f66d00d8b069c7daced,2014-06-14 00:21:19
3,greatmusic,2.4,1.0,954234692,6d45097acaaf6ed5ee55041f53249fa0,2014-10-22 18:33:52
4,greatmusic,2.4,1.0,195485165,6d45097acaaf6ed5ee55041f53249fa0,2014-10-22 18:36:54


In [65]:
#Confirm null values in new dataframe
df_sentiment.apply(lambda x: x.isnull().sum())

hashtag            0
sentiment_score    0
ol_avg             0
user_id            0
track_id           0
created_at         0
dtype: int64

In [66]:
df_sentiment.shape

(138021, 6)

In [67]:
df_sentiment.hashtag.value_counts().head(10)

postpunk           25299
punk               24730
deathrock          24699
tophits            22836
doommetal           4296
freshoutthebox      3997
realclassicrock     2473
slamhardstyle       2331
freshlysqueezed     2154
chill               2112
Name: hashtag, dtype: int64

Now that the two CSV files are joined (inner join), the new dataframe `df_sentiment` is reduced to **5,126,717** rows (from 17,560,114).

## Load the third dataset
* Remove tracks that were played less than 50 times
* Remove unnecessary columns
* Remove null values
* Reduce the dataset to English only language
* Merge it with the df_sentiment dataset based on `track_id`, `created_at` and `user_id` columns

In [84]:
#Load third dataset and limit it to only load 22 columns
df3 = pd.read_csv('../data/chelseapower/fixed_context_content_features.csv')

#Look at size of the dataset
df3.shape

(11614671, 22)

In [85]:
# Get the count of the track_id
counts = df3['track_id'].value_counts()

# Select the items where the track_id count is less than 50 and remove them
df3 = df3[~df3['track_id'].isin(counts[counts < 50].index)]

# Show info
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9143294 entries, 1 to 11614670
Data columns (total 22 columns):
 #   Column            Dtype  
---  ------            -----  
 0   coordinates       object 
 1   instrumentalness  float64
 2   liveness          float64
 3   speechiness       float64
 4   danceability      float64
 5   valence           float64
 6   loudness          float64
 7   tempo             float64
 8   acousticness      float64
 9   energy            float64
 10  mode              float64
 11  key               float64
 12  artist_id         object 
 13  place             object 
 14  geo               object 
 15  tweet_lang        object 
 16  track_id          object 
 17  created_at        object 
 18  lang              object 
 19  time_zone         object 
 20  user_id           float64
 21  id                int64  
dtypes: float64(12), int64(1), object(9)
memory usage: 1.6+ GB


By removing tracks that were played less than 50 times, the dataset is reduced from 11,614,671 to **9,143,294** rows. The following unnecessary columns will be droped:

- **coordinates**: no valuable information, will use `time_zone` instead
- **id**: no valuable information, will use `user_id instead
- **place**: no valuable information, will use `time_zone` instead
- **geo**: no valuable information, will use `time_zone` instead

In [86]:
#Drop unnecessary columns before merging with df_sentiment dataframe
df3 = df3.drop(['coordinates','id','place','geo'], axis=1)
df3.head()

,instrumentalness,liveness,speechiness,danceability,valence,loudness,tempo,acousticness,energy,mode,key,artist_id,tweet_lang,track_id,created_at,lang,time_zone,user_id
1,0.017700,0.0638,0.0624,0.769,0.752,-8.252,95.862,0.2670,0.826,1.0,7.0,5cddcd0e314e2f2223ab21937d2c8778,en,da3110a77b724072b08f231c9d6f7534,2014-01-01 05:54:22,en,NaN,2.205687e+09
2,0.000000,0.0860,0.0436,0.675,0.775,-4.432,97.030,0.2170,0.885,0.0,1.0,e41273f43af504714d85465294f1f369,en,ba84d88c10fb0e42d4754a27ead10546,2014-01-01 05:54:22,es,Mountain Time (US & Canada),1.325884e+08
3,0.000000,0.1430,0.0292,0.324,0.333,-5.647,74.101,0.2390,0.574,1.0,7.0,557ce373bd29743eb00a3723ab19ebe8,en,33f95122281f76e7134f9cbea3be980f,2014-01-01 05:54:24,en,Eastern Time (US & Canada),9.767522e+07
5,0.000000,0.1100,0.0375,0.641,0.912,-4.271,93.010,0.0268,0.787,1.0,0.0,f965ec352eb8c0efc0af46244754942f,en,8bd5206b84c968eda0af8bc86d6ab1d1,2014-01-01 05:54:25,en,Central Time (US & Canada),4.522857e+08
6,0.000006,0.3620,0.1340,0.554,0.677,-4.271,126.045,0.0216,0.878,1.0,3.0,a3abd1e016fdba82a91379b7e2b9ab69,en,23ced06ca57d37fa749b1595bc7ed1a4,2014-01-01 05:54:28,en,Quito,6.508628e+07


In [87]:
df3.apply(lambda x: x.isnull().sum())

instrumentalness       2601
liveness               2601
speechiness            3158
danceability           3158
valence                3976
loudness                  0
tempo                     0
acousticness           2601
energy                 2601
mode                   2601
key                    2601
artist_id                 0
tweet_lang                0
track_id                  0
created_at                0
lang                      0
time_zone           2689681
user_id               38025
dtype: int64

In [88]:
#Drop all null value rows
df3 = df3.dropna(subset=
                 ['instrumentalness', 'liveness', 'speechiness', 'danceability ',
                  'valence ', 'acousticness ', 'energy ', 'mode ', 'key ',
                  'time_zone ', 'user_id '])
#Convert mode to Int64
df3['mode'] = df3['mode'].astype('Int64')

df3.shape

KeyError: ['danceability ', 'valence ', 'acousticness ', 'energy ', 'mode ', 'key ', 'time_zone ', 'user_id ']

In [ ]:
df3.apply(lambda x: x.isnull().sum())

Reduced the dataset from 10,887,911 to **6,413,576** by dropping all null value rows.

In [72]:
#Limit dataset to only en (English) language
df3 = df3.loc[~((df3['lang'] != 'en')),:]
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4916702 entries, 3 to 11614670
Data columns (total 18 columns):
 #   Column            Dtype  
---  ------            -----  
 0   instrumentalness  float64
 1   liveness          float64
 2   speechiness       float64
 3   danceability      float64
 4   valence           float64
 5   loudness          float64
 6   tempo             float64
 7   acousticness      float64
 8   energy            float64
 9   mode              Int64  
 10  key               float64
 11  artist_id         object 
 12  tweet_lang        object 
 13  track_id          object 
 14  created_at        object 
 15  lang              object 
 16  time_zone         object 
 17  user_id           float64
dtypes: Int64(1), float64(11), object(6)
memory usage: 717.4+ MB


In [73]:
#Confirm change by looking at the unique values in the lang column
df3.lang.unique()

array(['en'], dtype=object)

Reduced the dataset from 7,740,906 to **4,916,702** by limiting it to English only (lang = en).

### Merge datasets based on track_id, created_at, and user_id

In [74]:
#Merge df_sentiment and df3 CSV files into new CSV file based on track_id, created_at and user_id
df4 = df_sentiment.merge(df3, on=['track_id','created_at','user_id'], how='inner')
df4.head()

,hashtag,sentiment_score,ol_avg,user_id,track_id,created_at,instrumentalness,liveness,speechiness,danceability,...,loudness,tempo,acousticness,energy,mode,key,artist_id,tweet_lang,lang,time_zone
0,greatmusic,2.4,1.0,252330820,8f2ac86abb8bd48273c8fc95b632e347,2014-02-13 16:18:51,0.000000,0.1980,0.1690,0.461,...,-9.046,143.055,0.100000,0.628,0,4.0,1a17d5f8af99355edd8a92679a02cb0d,en,en,Eastern Time (US & Canada)
1,greatmusic,2.4,1.0,29235188,45bbd6d7cd65dc77596af8c5c0b89a70,2014-05-15 20:38:46,0.000010,0.0454,0.0451,0.677,...,-13.237,77.837,0.031300,0.544,0,11.0,4a4e2ab094a4521b06252e9fdaf1fd0a,en,en,Eastern Time (US & Canada)
2,classic,1.0,1.0,29235188,45bbd6d7cd65dc77596af8c5c0b89a70,2014-05-15 20:38:46,0.000010,0.0454,0.0451,0.677,...,-13.237,77.837,0.031300,0.544,0,11.0,4a4e2ab094a4521b06252e9fdaf1fd0a,en,en,Eastern Time (US & Canada)
3,greatmusic,2.4,1.0,195485165,6d45097acaaf6ed5ee55041f53249fa0,2014-10-22 18:36:54,0.000000,0.0769,0.0427,0.591,...,-6.315,100.063,0.017100,0.746,1,11.0,f78fb5a7ddce990521f685522f3f8fce,en,en,London
4,amonamarth,0.3,0.0,124052165,d53b2de022666067050dae8f6645edc2,2014-09-26 16:06:48,0.000142,0.1130,0.0709,0.355,...,-4.054,130.427,0.000004,0.980,0,7.0,942c9f2520684c22eb6216a92b711f9e,en,en,Alaska


In [75]:
#Convert hashtag info string
df4['hashtag'] = df4['hashtag'].astype(str)

#Convert user_id info string
df4['user_id'] = df4['user_id'].astype(str)

#Show changes
df4.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 69428 entries, 0 to 69427
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   hashtag           69428 non-null  object 
 1   sentiment_score   69428 non-null  float64
 2   ol_avg            69428 non-null  float64
 3   user_id           69428 non-null  object 
 4   track_id          69428 non-null  object 
 5   created_at        69428 non-null  object 
 6   instrumentalness  69428 non-null  float64
 7   liveness          69428 non-null  float64
 8   speechiness       69428 non-null  float64
 9   danceability      69428 non-null  float64
 10  valence           69428 non-null  float64
 11  loudness          69428 non-null  float64
 12  tempo             69428 non-null  float64
 13  acousticness      69428 non-null  float64
 14  energy            69428 non-null  float64
 15  mode              69428 non-null  Int64  
 16  key               69428 non-null  float6

In [76]:
#Create new column sentiment that will be the predictor based on the sentiment_score values
df4['sentiment'] = np.where(df4['sentiment_score']>= 0.01, 1, 0)
df4.head()
#Confirm null values in new dataframe


hashtag            0
sentiment_score    0
ol_avg             0
user_id            0
track_id           0
created_at         0
dtype: int64

In [77]:
df4.apply(lambda x: x.isnull().sum())

hashtag             0
sentiment_score     0
ol_avg              0
user_id             0
track_id            0
created_at          0
instrumentalness    0
liveness            0
speechiness         0
danceability        0
valence             0
loudness            0
tempo               0
acousticness        0
energy              0
mode                0
key                 0
artist_id           0
tweet_lang          0
lang                0
time_zone           0
sentiment           0
dtype: int64

The following unnecessary columns will be droped:

- **hashtag**: no valuable information, will use `sentiment_score` instead
- **created_at**: no valuable information after the data merge
- **artist_id**: no valuable information for this model
- **tweet_lang**: no valuable information for this model
- **lang**: no valuable information since this dataset has been reduced to English only

In [ ]:

#Drop unnecessary columns lang and created_at
df4 = df4.drop(['hashtag','created_at','artist_id','tweet_lang','lang'], axis=1)

df4.head()

In [ ]:
#Look at unique values for time_zone column
df4.time_zone.unique()

In [ ]:
#Make all USA Timezone values consistent
df4['time_zone'].replace('Eastern Time (US & Canada)', 'Eastern Time',inplace=True)
df4['time_zone'].replace('Central Time (US & Canada)', 'Central Time',inplace=True)
df4['time_zone'].replace('Pacific Time (US & Canada)', 'Pacific Time',inplace=True)
df4['time_zone'].replace('Mountain Time (US & Canada)', 'Mountain Time',inplace=True)
df4['time_zone'].replace('Alaska', 'Alaska Time',inplace=True)
df4['time_zone'].replace('Hawaii', 'Hawaii Time',inplace=True)
df4['time_zone'].replace('Arizona', 'Mountain Time',inplace=True)
df4['time_zone'].replace('America/Chicago', 'Central Time',inplace=True)
df4['time_zone'].replace('America/New_York', 'Eastern Time',inplace=True)
df4['time_zone'].replace('America/Los_Angeles', 'Pacific Time',inplace=True)
df4['time_zone'].replace('America/Denver', 'Mountain Time',inplace=True)
df4['time_zone'].replace('America/Detroit', 'Eastern Time',inplace=True)

In [ ]:
#Limit dataset to only USA time zones
df4 = df4.loc[~((df4['time_zone'] != 'Eastern Time') & (df4['time_zone'] != 'Central Time') & (df4['time_zone'] != 'Pacific Time') & 
               (df4['time_zone'] != 'Mountain Time') & (df4['time_zone'] != 'Alaska Time') & (df4['time_zone'] != 'Hawaii Time')),:]
df4.info()

Reduced the dataset from 4,916,702 to **2,267,492** by limiting it to USA Timezone data only.

### Create MVP Dataset
- Reduced to U.S.A. only dataset
- **13 feature columns**
    - sentiment_score: Sentiment score from Opinion Lexicon. If no score was provided then a 0 was input.
    - user_id: Unique user ID
    - track_id: Unique track ID
    - sentiment: Sentiment extracted from sentiment score based on range 0-1
    - Instrumentalness: Signifies whether a track contains vocals.
    - Liveness: Presence of an audience in the track recording (range is [0, 1], where 1 indicates high probability of liveness).
    - Speechiness: Presence of spoken words in a track - whether a track contains more music or words (range is [0, 1], where 0 is a track with no speech).
    - Danceability: Suitability of a track for dancing based on a combination of musical elements like tempo, rhythm stability, beat strength, and overall regularity (range is [0, 1], where 1 is a most danceable song).
    - Valence: Musical positiveness conveyed by a track (range is [0, 1], where 1 is a highly positive and cheerful song).
    - Loudness: The overall loudness of a track in decibel (dB).
    - Tempo: The overall estimated tempo of a track in beats per minute (BPM).
    - Acousticness: Probability whether a track is acoustic (range is [0, 1]).
    - Energy: Perceptual measure of intensity and activity (range is [0, 1], where 1 indicates a high-energy track).
    - Mode: Modality (major or minor) of a track, i.e., the type of scale from which its melodic content is derived. Major is 1 and minor is 0.
    - Key: The key that the track is in. Integers map to pitches using standard Pitch Class notation.
    - tz_Alaska_Time: Categorical - One hot encoded from (dropped) timezone column
    - tz_Central_Time: Categorical - One hot encoded from (dropped) timezone column
    - tz_Eastern_Time: Categorical - One hot encoded from (dropped) timezone column
    - tz_Hawaii_Time: Categorical - One hot encoded from (dropped) timezone column
    - tz_Mountain_Time: Categorical - One hot encoded from (dropped) timezone column
    - tz_Pacific_Time: Categorical - One hot encoded from (dropped) timezone column
    - sentiment: Extracted from `sentiment_score` with a range 0-1 and is the **predictor column**

In [ ]:
# Reorder columns
df_mvp = df4[['sentiment','sentiment_score','user_id','track_id','time_zone','instrumentalness',
              'liveness','speechiness','danceability','valence','loudness','tempo','acousticness','energy','mode','key']]
df_mvp.info()

In [ ]:
# Create new dataset with only user_id, track_id and time_zone and the category variables
df_timezone = df_mvp.drop(['user_id','track_id','sentiment','sentiment_score','instrumentalness','liveness','speechiness',
                             'danceability','valence','loudness','tempo','acousticness','energy','mode','key'], axis=1)
df_timezone.head()

In [ ]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# One Hot Encode the category data set
one_hot_df = pd.get_dummies(df_timezone)
one_hot_df.head()

In [ ]:
# Rename the columns
one_hot_df.columns = ['tz_Alaska_Time','tz_Central_Time','tz_Eastern_Time','tz_Hawaii_Time','tz_Mountain_Time','tz_Pacific_Time']
one_hot_df.head()

In [ ]:
#Drop time_zone from MVP dataset
df_mvp = df_mvp.drop(["time_zone"], axis=1)

In [ ]:
#Concatenate one hot encoded dataframe
df_mvp = pd.concat([df_mvp,one_hot_df], axis=1)
df_mvp.head()

In [ ]:
df_mvp.to_csv('module4_cleaned.csv',index=False)

## Explore the data

* Look at the distribution for the data
* Look for Multicollinearity
* Remove unnecessary features
* Balace and scale data

In [ ]:
#Look at value counts of the predictor variable sentiment
df_mvp.sentiment.value_counts()

In [ ]:
# Visualize the predictor variable
import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(x='sentiment', data=df_mvp, palette='hls')
plt.show()

**Observation**: The data is very imbalanced and will need to be balanced.

In [ ]:
# Create continuous dataset and look at distributions of data
df_mvp_lin = df_mvp.drop(['user_id','track_id','sentiment','mode','tz_Alaska_Time','tz_Central_Time','tz_Eastern_Time',
                          'tz_Hawaii_Time','tz_Mountain_Time','tz_Pacific_Time'], axis=1)
df_mvp_lin.hist(figsize = [30, 20]);

In [ ]:
#Create coorelation heatmap and check for multicolinarity
from matplotlib import pyplot as plt
import seaborn as sns

correlation = df_mvp_lin.corr()
plt.figure(figsize=(14, 12))
heatmap = sns.heatmap(correlation, annot=True, linewidths=0, vmin=-1, cmap="RdBu_r")

**Observation**: Loudness and Energy seem to be highly coorelated.

## Logistic Regression
* Normalize the data prior to fitting the model
* Train-Test Split
* Fit the model
* Predict
* Evaluate

In [ ]:
# Define X and y
y = df_mvp['sentiment']
X = df_mvp.drop('sentiment', axis = 1)

In [ ]:
# Normalizing the data prior to fitting the model.
x_feats = ['sentiment_score','instrumentalness','liveness','speechiness','danceability',
           'loudness','tempo','acousticness','energy','mode','key','valence','tz_Alaska_Time',
           'tz_Central_Time','tz_Eastern_Time','tz_Hawaii_Time','tz_Mountain_Time','tz_Pacific_Time']

X = pd.get_dummies(df_mvp[x_feats], drop_first=False)
y = df_mvp.sentiment
X.head()

In [ ]:
from sklearn.model_selection import train_test_split

# Splitting the data into train and test sets (automatically uses stratified sampling by labels)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(fit_intercept = True, C=1e12)
model_log = logreg.fit(X_train, y_train)
model_log

In [ ]:
print(y_train.value_counts())
print(y_test.value_counts())

In [ ]:
#Predict against test set using Sigmoid function
import time
start = time.time()

y_hat_test = logreg.predict(X_test)
y_hat_train = logreg.predict(X_train)

end = time.time()
print("Execution time:", end - start)

In [ ]:
y_hat_test = logreg.predict_proba(X_test)
y_hat_test[0]

In [ ]:
import time
start = time.time()

logreg.predict_proba(X_train)

end = time.time()
print("Execution time:", end - start)

In [ ]:
# How may times was the classifier correct for the training set?
logreg.score(X_train, y_train)

In [ ]:
# How may times was the classifier correct for the test set?
logreg.score(X_test, y_test)

### Classification Model Performance
Check the precision, recall, and accuracy of the model.

In [ ]:
# Function to calculate the precision
def precision(y_hat, y):
    y_y_hat = list(zip(y, y_hat))
    tp = sum([1 for i in y_y_hat if i[0]==1 and i[1]==1])
    fp = sum([1 for i in y_y_hat if i[0]==0 and i[1]==1])
    return tp/float(tp+fp)

In [ ]:
# Function to calculate the recall
def recall(y_hat, y):
    y_y_hat = list(zip(y, y_hat))
    tp = sum([1 for i in y_y_hat if i[0]==1 and i[1]==1])
    fn = sum([1 for i in y_y_hat if i[0]==1 and i[1]==0])
    return tp/float(tp+fn)

In [ ]:
# Function to calculate the accuracy
def accuracy(y_hat, y):
    y_y_hat = list(zip(y, y_hat))
    tp = sum([1 for i in y_y_hat if i[0]==1 and i[1]==1])
    tn = sum([1 for i in y_y_hat if i[0]==0 and i[1]==0])
    return (tp+tn)/float(len(y_hat))

In [ ]:
# Calculate the precision, recall and accuracy of the classifier.
y_hat_test = logreg.predict(X_test)
y_hat_train = logreg.predict(X_train)

print('Training Precision: ', precision(y_hat_train, y_train))
print('Testing Precision: ', precision(y_hat_test, y_test))
print('\n')

print('Training Recall: ', recall(y_hat_train, y_train))
print('Testing Recall: ', recall(y_hat_test, y_test))
print('\n')

print('Training Accuracy: ', accuracy(y_hat_train, y_train))
print('Testing Accuracy: ', accuracy(y_hat_test, y_test))

### Resample data since it's imbalanced and all scores are very high.

In [ ]:
# concatenate our training data back together
training_data = pd.concat([X_train, y_train], axis=1)

In [ ]:
# separate minority and majority classes
not_safe = training_data[training_data.sentiment==0]
safe = training_data[training_data.sentiment==1]

In [ ]:
from sklearn.utils import resample
# upsample minority
not_safe_upsampled = resample(not_safe, 
                              replace=True, # sample with replacement
                              n_samples=len(safe), # match number in majority class
                              random_state=42) # reproducible results

In [ ]:
# combine majority and upsampled minority
upsampled = pd.concat([safe, not_safe_upsampled])

In [ ]:
# check new class counts
print(upsampled.sentiment.value_counts())

In [ ]:
sns.countplot(x='sentiment', data=upsampled, palette='hls')
plt.show()

In [ ]:
X_train = upsampled.drop('sentiment', axis=1)
y_train = upsampled.sentiment

In [ ]:
# Scaling X using StandardScaler
from sklearn.preprocessing import StandardScaler

# Only fit training data to avoid data leakage
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=list(X.columns))
X_test = pd.DataFrame(scaler.transform(X_test), columns=list(X.columns))
X_train.head()

### Run another Logistic Regression Model with resampled data

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

logreg2 = LogisticRegression(fit_intercept = True, C=1e12)
model_log = logreg2.fit(X_train, y_train)
model_log

In [ ]:
print(y_train.value_counts())
print(y_test.value_counts())

In [ ]:
# Predict against test set using Sigmoid function
import time
start = time.time()

y_hat_test = logreg2.predict(X_test)
y_hat_train = logreg2.predict(X_train)

end = time.time()
print("Execution time:", end - start)

In [ ]:
y_hat_test = logreg2.predict_proba(X_test)
y_hat_test[0]

In [ ]:
import time
start = time.time()

logreg2.predict_proba(X_train)

end = time.time()
print("Execution time:", end - start)

In [ ]:
#Train score
logreg2.score(X_train, y_train)

In [ ]:
#Test score
logreg2.score(X_test, y_test)

In [ ]:
logreg2.coef_[0]

In [ ]:
for feature, weight in zip(X.columns, logreg2.coef_[0]):
    print("{} has a weight of : {}".format(feature, weight))

### Confusion Matrix

Show the performance of the classification model.

In [ ]:
#Create a Confusion Matrix
from sklearn.metrics import confusion_matrix

cnf_matrix = confusion_matrix(y_train, y_hat_train)
print('Confusion Matrix:\n',cnf_matrix)

In [ ]:
#Plot the Confusion Matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

plt.imshow(cnf_matrix,  cmap=plt.cm.Blues) #Create the basic matrix

#Add title and axis labels
plt.title('Confusion Matrix')
plt.ylabel('True label')
plt.xlabel('Predicted label')

#Add appropriate axis scales
class_names = set(y) #Get class labels to add to matrix
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names, rotation=45)
plt.yticks(tick_marks, class_names)

#Add Labels to each cell
thresh = cnf_matrix.max() / 2. #Used for text coloring below
#Here we iterate through the confusion matrix and append labels to our visualization
for i, j in itertools.product(range(cnf_matrix.shape[0]), range(cnf_matrix.shape[1])):
        plt.text(j, i, cnf_matrix[i, j],
                 horizontalalignment="center",
                 color="white" if cnf_matrix[i, j] > thresh else "black")

#Add a side bar legend showing colors
plt.colorbar()

In [ ]:
#conf_matrix function
def conf_matrix(y_true, y_pred):
    cm = {'TP': 0, 'TN': 0, 'FP': 0, 'FN': 0}
    
    for ind, label in enumerate(y_true):
        pred = y_pred[ind]
        if label == 1:
            # CASE: TP 
            if label == pred:
                cm['TP'] += 1
            # CASE: FN
            else:
                cm['FN'] += 1
        else:
            # CASE: TN
            if label == pred:
                cm['TN'] += 1
            # CASE: FP
            else:
                cm['FP'] += 1
    return cm

In [ ]:
#Set variable
model_confusion_matrix = conf_matrix(y_train, y_hat_train)

In [ ]:
#Precision function
def precision(confusion_matrix):
    return confusion_matrix['TP'] / (confusion_matrix['TP'] + confusion_matrix['FP'])

In [ ]:
#Recall function
def recall(confusion_matrix):
    return confusion_matrix['TP'] / (confusion_matrix['TP'] + confusion_matrix['FN'])

In [ ]:
#Accuracy function
def accuracy(confusion_matrix):
    return (confusion_matrix['TP'] + confusion_matrix['TN']) / sum(confusion_matrix.values())

In [ ]:
#f1 score
def f1(confusion_matrix):
    precision_score = precision(confusion_matrix)
    recall_score = recall(confusion_matrix)
    numerator = precision_score * recall_score
    denominator = precision_score + recall_score
    return 2 * (numerator / denominator)

In [ ]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

preds = [y_hat_train]

for ind, i in enumerate(preds):
    print('-'*40)
    print('Model Metrics:'.format(ind + 1))
    print('Precision: {}'.format(precision_score(y_train, i)))
    print('Recall: {}'.format(recall_score(y_train, i)))
    print('Accuracy: {}'.format(accuracy_score(y_train, i)))
    print('F1-Score: {}'.format(f1_score(y_train, i)))

In [ ]:
from sklearn.metrics import classification_report

for ind, i in enumerate(preds):
    print('-'*40)
    print("Model Classification Report:".format(ind + 1))
    print(classification_report(y_train, i))

## Cross Validation
Repeat a train-test-split creation 20 times, using a test_size of 0.05.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
import matplotlib.pyplot as plt

num = 20
train_err = []
test_err = []
for i in range(num):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)
    logreg2.fit(X_train, y_train)
    y_hat_train = logreg2.predict(X_train)
    y_hat_test = logreg2.predict(X_test)
    train_err.append(mean_squared_error(y_train, y_hat_train))
    test_err.append(mean_squared_error(y_test, y_hat_test))
plt.scatter(list(range(num)), train_err, label='Training Error')
plt.scatter(list(range(num)), test_err, label='Testing Error')
plt.legend();

In [ ]:
#K-Fold Cross Validation
from sklearn.model_selection import cross_val_score

cv_5_results = np.mean(cross_val_score(logreg2, X, y, cv=5, scoring="accuracy"))
cv_10_results = np.mean(cross_val_score(logreg2, X, y, cv=10, scoring="accuracy"))
cv_20_results = np.mean(cross_val_score(logreg2, X, y, cv=20, scoring="accuracy"))

print(cv_5_results)
print(cv_10_results)
print(cv_20_results)

## Create a Sequential Neural Network
- ReLU activation function
- Sigmoid function on the output layer 

In [ ]:
# Load libraries
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint

# split into input (X) and output (y) variables
X = pd.get_dummies(df_mvp[x_feats], drop_first=False)
y = df_mvp.sentiment

In [ ]:
# define the keras model
model = Sequential()
model.add(Dense(12, input_dim=18, activation='relu'))
model.add(Dense(19, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Set callback functions to early stop training and save the best model so far
checkpoint = [ModelCheckpoint(filepath='models.hdf5', save_best_only=True, monitor='val_loss')]

In [ ]:
# fit the keras model on the dataset
model.fit(X, y, epochs=10, callbacks=checkpoint, batch_size=100)

In [ ]:
# evaluate the keras model
_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))

## Create a Sequential Neural Network with the scaled data
- ReLU activation function
- Sigmoid function on the output layer 

In [ ]:
# split into input (X) and output (y) variables
X = pd.get_dummies(X_train, drop_first=False) #X_train = upsampled.drop('sentiment', axis=1)
y = y_train #y_train = upsampled.sentiment

In [ ]:
# define the keras model
model2 = Sequential()
model2.add(Dense(12, input_dim=18, activation='relu'))
model2.add(Dense(8, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))

In [ ]:
# compile the keras model
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Set callback functions to early stop training and save the best model so far
checkpoint = [ModelCheckpoint(filepath='models.hdf5', save_best_only=True, monitor='val_loss')]

In [ ]:
# fit the keras model on the dataset
model2.fit(X, y, epochs=10, callbacks=checkpoint, batch_size=100)

In [ ]:
# evaluate the keras model
_, accuracy = model2.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))